In [1]:
import extract
import pandas as pd
import os
import geopandas as gpd
import numpy as np

In [2]:
import importlib
importlib.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [3]:
# open state_fips
state_fips = pd.read_csv('state_fips.csv')

In [4]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [10]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data\Santa Fe Council Districts\CouncilDistricts.shp')
districts.columns

Index(['OBJECTID_1', 'OBJECTID_2', 'OBJECTID', 'CouncilDis', 'Councilor',
       'Councilor_', 'Acres', 'Shape_Leng', 'Shape_Le_1', 'Shape_Le_2',
       'Shape_Area', 'geometry'],
      dtype='object')

In [11]:
# only keep district and geometry
district_name = 'CouncilDis'
year=2018
desc='santa_fe_districts'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
# set district to only last char
districts['district'] = districts['district'].astype(int)

In [12]:
districts

,district,geometry
0,4,"POLYGON ((-105.930 35.637, -105.931 35.636, -1..."
1,2,"POLYGON ((-105.895 35.686, -105.895 35.686, -1..."
2,1,"POLYGON ((-105.895 35.686, -105.895 35.686, -1..."
3,3,"POLYGON ((-106.036 35.650, -106.036 35.649, -1..."


In [13]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)